In [1]:
# Insert the directory
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


In [2]:
from review_df import ReviewDF
df = ReviewDF.get_df()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [3]:
df.head()

,text,sentiment
0,I have bought several of the Vitality canned d...,1
1,Product arrived labeled as Jumbo Salted Peanut...,0
2,This is a confection that has been around a fe...,1
3,If you are looking for the secret ingredient i...,0
4,Great taffy at a great price There was a wide...,1


In [3]:
import sklearn
from sklearn.model_selection import train_test_split

# split set
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['sentiment'], test_size= .15, random_state= 111, stratify= None )
x_train, x_val, y_train, y_val= train_test_split(x_train, y_train, test_size= .25, random_state= 111, stratify= None )

In [4]:
print(f'train set Count: {len(x_train)}')
print(f'validation set Count: {len(x_val)}')
print(f'test set Count: {len(x_test)}')

train set Count: 188991
validation set Count: 62997
test set Count: 44469


In [5]:
import tensorflow as tf

from tensorflow import keras
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow import keras

import re
import string
import random

unique_words = 15000

vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=unique_words,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='int',
    output_sequence_length=110,
    pad_to_max_tokens=False,
    vocabulary=None,
    idf_weights=None,
    sparse=False,
    ragged=False)


In [6]:
vectorize_layer.adapt(x_train)

In [7]:
vocab = vectorize_layer.get_vocabulary() 
print("vocabulary =", len(vocab)," entries")
print("vocabulary first 10 entries")
for word in range(10):
  print(word, " represents the word: ", vocab[word])

for i in x_train[:2]:
  print(" Given raw data: " )
  print(i)
  tokenized = vectorize_layer(tf.expand_dims(i, -1))
  print(" Tokenized & Transformed to int vectors: " )
  print (tokenized)
  print("text after tokenize & transform: ")
  transformedText = ""
  for each in tf.squeeze(tokenized):
    transformedText= transformedText+ " "+ vocab[each]
  print(transformedText)

vocabulary = 15000  entries
vocabulary first 10 entries
0  represents the word:  
1  represents the word:  [UNK]
2  represents the word:  the
3  represents the word:  i
4  represents the word:  and
5  represents the word:  a
6  represents the word:  it
7  represents the word:  to
8  represents the word:  this
9  represents the word:  is
 Given raw data: 
Ive been making wine for about a year and recently tried this one Following others advice I made this a 5 gallon batch I also added raisins peppercorns and oak chips Although its only been bottled a month this is the  flavor Ive been trying to get dry oaky and a hint of pepper Ill be making more soon
 Tokenized & Transformed to int vectors: 
tf.Tensor(
[[  91   85  332  923   11   78    5  276    4  600   79    8   39 2104
   391 2871    3  121    8    5  264 1730  787    3   86  323 1039 3312
     4 5469  309  476   35   67   85 1916    5  480    8    9    2   41
    91   85  313    7   58  350    1    4    5  733   10  648  339   38


In [8]:
filepath = "vectorize_layer_model_imbalance"

vectorize_layer_model = tf.keras.models.Sequential()
vectorize_layer_model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
vectorize_layer_model.add(vectorize_layer)
vectorize_layer_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 110)              0         
 torization)                                                     
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [9]:
vectorize_layer_model.save(filepath, save_format="tf")

INFO:tensorflow:Assets written to: vectorize_layer_model_imbalance/assets


In [10]:
loaded_vectorize_layer_model = tf.keras.models.load_model(filepath)
loaded_vectorize_layer = loaded_vectorize_layer_model.layers[0]

In [11]:
def convert_text_input(sample):
    text = sample
    text = tf.expand_dims(text, -1)  
    return tf.squeeze(loaded_vectorize_layer(text))

In [12]:
train_seq = convert_text_input(x_train)
val_seq = convert_text_input(x_val)
test_seq = convert_text_input(x_test)

In [43]:
train_seq.shape

TensorShape([188991, 110])

In [36]:
from keras import Sequential, regularizers 

from keras.layers import Embedding, LSTM, Dense
from keras import losses

embedding_dim = 64
sequence_length = 110
cells = 16
batch_size= 64
epochs = 20

In [37]:
lstm = keras.Sequential(
   [ Embedding(unique_words+1, embedding_dim, input_length=sequence_length) ,
  LSTM(cells),
  Dense(32,activation='relu'),
  Dense(1,activation='sigmoid')]
)

In [38]:
lstm.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 110, 64)           960064    
                                                                 
 lstm_3 (LSTM)               (None, 16)                5184      
                                                                 
 dense_9 (Dense)             (None, 32)                544       
                                                                 
 dense_10 (Dense)            (None, 1)                 33        
                                                                 
Total params: 965,825
Trainable params: 965,825
Non-trainable params: 0
_________________________________________________________________


In [39]:
metrics = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision & recall curve
]
lstm.compile(loss='binary_crossentropy',optimizer=keras.optimizers.Adam(1e-3),metrics=metrics)

In [40]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

earlyStopping = EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')

lstm.fit(train_seq,y_train,validation_data= (val_seq, y_val),
                    epochs= epochs,batch_size= batch_size, verbose=1, callbacks=[earlyStopping, mcp_save])

Epoch 1/20
2953/2953 [==============================] - 225s 75ms/step - loss: 0.3712 - tp: 160986.0000 - fp: 24089.0000 - tn: 2130.0000 - fn: 1786.0000 - accuracy: 0.8631 - precision: 0.8698 - recall: 0.9890 - auc: 0.6804 - prc: 0.9201 - val_loss: 0.3644 - val_tp: 54223.0000 - val_fp: 8774.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.8607 - val_precision: 0.8607 - val_recall: 1.0000 - val_auc: 0.7494 - val_prc: 0.9458
Epoch 2/20
2953/2953 [==============================] - 212s 72ms/step - loss: 0.3406 - tp: 158715.0000 - fp: 20681.0000 - tn: 5538.0000 - fn: 4057.0000 - accuracy: 0.8691 - precision: 0.8847 - recall: 0.9751 - auc: 0.7492 - prc: 0.9361 - val_loss: 0.3701 - val_tp: 54223.0000 - val_fp: 8774.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.8607 - val_precision: 0.8607 - val_recall: 1.0000 - val_auc: 0.6492 - val_prc: 0.9259
Epoch 3/20
2953/2953 [==============================] - 212s 72ms/step - loss: 0.3899 - tp: 162772.0000 - fp: 2621

In [41]:
file_name = "/content/drive/MyDrive/Colab Notebooks/LSTM_imbalanced.pkl"  

In [42]:
tf.keras.models.save_model(lstm, file_name)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/LSTM_imbalanced.pkl/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/LSTM_imbalanced.pkl/assets


In [43]:
lstm = tf.keras.models.load_model(file_name)

In [44]:
lstm_pred = lstm.predict(test_seq)
lstm_y = lstm_pred.flatten()
lstm_y
lstm_y = np.where(lstm_y > 0.5, 1, 0)
lstm_y

array([1, 1, 1, ..., 1, 1, 1])

In [45]:
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score


print('Accuracy:', accuracy_score(y_test, lstm_y))
print('F1 score:', f1_score(y_test, lstm_y))
print('Recall:', recall_score(y_test, lstm_y))
print('Precision:', precision_score(y_test, lstm_y))

print(classification_report(y_test, lstm_y))

Accuracy: 0.861701409970991
F1 score: 0.9257138715755906
Recall: 1.0
Precision: 0.861701409970991
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      6150
           1       0.86      1.00      0.93     38319

    accuracy                           0.86     44469
   macro avg       0.43      0.50      0.46     44469
weighted avg       0.74      0.86      0.80     44469



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
weighted_results = lstm.evaluate(test_seq, y_test,
                                           batch_size=64, verbose=0)
for name, value in zip(lstm.metrics_names, weighted_results):
  print(name, ': ', value)
print()

loss :  0.19164100289344788
tp :  36886.0
fp :  1806.0
tn :  4344.0
fn :  1433.0
accuracy :  0.927162766456604
precision :  0.9533236622810364
recall :  0.9626033902168274
auc :  0.9482412934303284
prc :  0.9886367321014404

